# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [3]:
# Importing necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

In [4]:
# Your code here
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [5]:
df.info()
# No null values
# 4 columns
# 8188 entries in df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [6]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [15]:
viewed = set(df[df['action']=='view']['id'].unique())
clicked = set(df[df['action']=='click']['id'].unique())
print('Viewed: ', len(viewed))
print('Clicked: ', len(clicked))
print('Clicked, Did Not View: ', len(clicked-viewed))
# All those who clicked also viewed

Viewed:  6328
Clicked:  1860
Clicked, Did Not View:  0


In [13]:
df['group'].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [17]:
control = set(df[df['group']=='control']['id'].unique())
experiment = set(df[df['group']=='experiment']['id'].unique())
print('Control Group: ', len(control))
print('Experimental Group: ', len(experiment))
print('Overlap: ', len(control & experiment))
# No overlap between C and E groups

Control Group:  3332
Experimental Group:  2996
Overlap:  0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [23]:
# Your code here
# Test to determine if experimental homepage was more effective than control group
# Use Welch's t-test since sample sizes are not equal
# Effectivity parameter is number of clicks

df['count'] = 1 # Adding 'count' column so that statistics can be done
df.head() # Checking to see if new column was added to df

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [26]:
# Creating binary variable for clicks ('1' if 'yes' and '0' if 'no')

# Use .pivot method to reshape pandas df
control = df[df['group']=='control'].pivot(index='id', columns='action', values='count')
# Use .fillna method to replace null values with 0
control = control.fillna(value=0)

experiment = df[df['group']=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

print('Total Clicks: ', (control['click'].sum(), experiment['click'].sum()))
print('Mean Clicks: ', (control['click'].mean(), experiment['click'].mean()))
# The experiment group had more mean clicks (0.31) than the control group (0.28)
# Use statistical test to determine significance in mean difference

Total Clicks:  (932.0, 928.0)
Mean Clicks:  (0.2797118847539016, 0.3097463284379172)


In [27]:
# Perform Welch's t-test

import flatiron_stats as fs # Welch's t-test functions stored here

fs.p_value_welch_ttest(control['click'], experiment['click'])

# Gives an error because lab is downloaded locally and cannot access flatiron_stats

ModuleNotFoundError: No module named 'flatiron_stats'

In [30]:
# Recycle functions from Welch's t-test lab to find p-value

import scipy.stats as stats

def welch_t(a, b):
    
    """ Calculate Welch's t-statistic for two samples. """
    numerator = a.mean() - b.mean()
    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    # variance is the std squared
    t_score = np.abs(numerator/denominator) #absolute value assures that a positive value will be returned
    return t_score # Return the t-score!

def welch_df(a, b):
    
    """ Calculate the effective degrees of freedom for two samples. """
    s1 = a.var(ddof=1) 
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    df = numerator/denominator
    return df # Return the degrees of freedom

def p_value(a, b, two_sided=False):
    t = welch_t(a, b) #using welch_t func to calculate t
    df = welch_df(a, b) #using welch_df func to calculate df
    
    p = 1-stats.t.cdf(np.abs(t), df) #calculating p-value given t and df
    
    if two_sided:
        return 2*p
    else:
        return p # Return the p-value!

In [31]:
p_value(control['click'], experiment['click'])

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [32]:
# Your code here
control_rate = control['click'].mean() # click rate of control group
expected_experiment_clicks = control_rate * len(experiment) # multiply control group click rate by experiment sample size
print(expected_experiment_clicks)
# This is the expected number of clicks of the experiment group if they had the same click rate as the control group

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [33]:
# Your code here
n = len(experiment) # sample size
p = control_rate # click rate of control group = probability
var = n * p * (1-p) # variance formula given above
std = np.sqrt(var) # std is the square root of the variance
print(std)

24.568547907005815


In [35]:
actual_experiment_clicks = experiment['click'].sum()

# Calculate z-score
z_score = (actual_experiment_clicks - expected_experiment_clicks)/std # z = (x - mu)/theta  
# z-score = (observed value - sample mean)/standard deviation
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [36]:
# Your code here

# Calculate p-value for z-score
p_val = 1 - stats.norm.cdf(z_score) 
print(p_val)

0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: The p-value of the Welch's t-test and the verification test are similar in that they both indicate significance at an alpha of 0.01. The Welch's t-test, however, did yield a larger p-value. For both tests, there is enough evidence to reject the null hypothesis.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.